In [1]:
import pandas as p
import numpy as np

In [3]:
df = p.read_csv('./data/bangkok_traffy.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787026 entries, 0 to 787025
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ticket_id      778254 non-null  object 
 1   type           786929 non-null  object 
 2   organization   786455 non-null  object 
 3   comment        778254 non-null  object 
 4   photo          786911 non-null  object 
 5   photo_after    641309 non-null  object 
 6   coords         787026 non-null  object 
 7   address        778254 non-null  object 
 8   subdistrict    786460 non-null  object 
 9   district       786465 non-null  object 
 10  province       786831 non-null  object 
 11  timestamp      787026 non-null  object 
 12  state          787026 non-null  object 
 13  star           274097 non-null  float64
 14  count_reopen   787026 non-null  int64  
 15  last_activity  787026 non-null  object 
dtypes: float64(1), int64(1), object(14)
memory usage: 96.1+ MB


,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


### Data Preprocessing

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer 

df_subset = df[['comment', 'type']].dropna().copy()

# Sample 100,000 rows for manageable processing
df_subset = df_subset.sample(n=100000, random_state=2024).reset_index(drop=True)

label_lists = (
    df_subset['type']
    .astype(str)
    .str.strip('{}')
    .str.split(',')
    .apply(lambda items: [item.strip() for item in items if item.strip()])
)
valid_mask = label_lists.map(bool)
comments = df_subset.loc[valid_mask, 'comment']
labels = label_lists.loc[valid_mask].tolist()

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(labels)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    comments, Y, test_size=0.2, random_state=2025
)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((68531,), (68531, 24), (17133,), (17133, 24))

### Classical Model
OneVsRestClassifier + LogisticRegression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier

clf = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=50000, ngram_range=(1, 2))),
    ('ovr', OneVsRestClassifier(LogisticRegression(max_iter=1000)))
])

clf.fit(X_train, Y_train)

Y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, f1_score

# Full Report
# print(classification_report(Y_test, Y_pred, target_names=mlb.classes_, zero_division=0))

# F1 score
macro_f1 = f1_score(Y_test, Y_pred, average='macro')
micro_f1 = f1_score(Y_test, Y_pred, average='micro')
weighted_f1 = f1_score(Y_test, Y_pred, average='weighted')

print(f"Macro F1 score: {macro_f1:.4f}")
print(f"Micro F1 score: {micro_f1:.4f}")
print(f"Weighted F1 score: {weighted_f1:.4f}")

Macro F1 score: 0.6625
Micro F1 score: 0.8020
Weighted F1 score: 0.7924


### Transformer-based Model
WangchanBERTa

In [14]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

import transformers
print(transformers.__version__)

4.57.2


In [7]:
# LOAD TOKENIZER

model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# tokenize function
def tokenize(batch_texts):
    return tokenizer(
        batch_texts,
        padding='max_length',
        truncation=True,
        max_length=128
    )

train_enc = tokenize(X_train.tolist())
test_enc  = tokenize(X_test.tolist())

In [8]:
# PYTORCH DATASET

class CommentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CommentDataset(train_enc, Y_train)
test_dataset  = CommentDataset(test_enc, Y_test)

In [ ]:
# METRICS FUNCTION FOR TRAINER EVALUATION 

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = (logits >= 0.5).astype(int)

    return {
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }

# MODEL AND TRAINING ARGUMENTS

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
# PREDICTION

preds = trainer.predict(test_dataset)
y_pred = (preds.predictions >= 0.5).astype(int)

In [ ]:
# EVALUATION REPORT

print("\n=== Classification Report ===")
print(classification_report(Y_test, y_pred, target_names=mlb.classes_, zero_division=0))

In [ ]:
# USE MODEL FOR NEW COMMENTS

def predict_comment(text):
    enc = tokenize([text])
    item = {k: torch.tensor(v) for k, v in enc.items()}

    with torch.no_grad():
        output = model(**item)
        pred = (output.logits.numpy() >= 0.5).astype(int)[0]

    labels = [mlb.classes_[i] for i in range(len(pred)) if pred[i] == 1]

    return labels

print(predict_comment("มีขยะเต็มทางเดิน น้ำเน่าในท่อระบายน้ำ"))